In [1]:
from tensorflow.keras.layers  import Dense, Activation, Conv2D, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import numpy as np
from skimage.transform import resize
import glob
import cv2
import zipfile

In [2]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"danielreyesdhats","key":"546f8b8a22d19b7001b860532f3a07c1"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d grassknoted/asl-alphabet

In [4]:
local_zip = "/kaggle/working/asl-alphabet.zip"
zip_ref = zipfile.ZipFile(local_zip, "r")
file_extract = "/kaggle/working/asl-alphabet"
zip_ref.extractall(file_extract) 
zip_ref.close()

In [5]:
features = []
labels = []
imageSize = 64
classes = ["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z", 'del','nothing','space']
n_classes=len(classes)
for i in range(len(classes)):
    file_list = glob.glob("/kaggle/working/asl-alphabet/asl_alphabet_train/asl_alphabet_train/" + classes[i] + "/*.jpg")
    for item in range(1000):
        features.append(file_list[item])
        labels.append([i])
print("Dataset Feature size : ",len(features))
print("Dataset labels size : ",len(labels))

In [6]:
features,labels = shuffle(features,labels,random_state=0)

In [7]:
data = []
for ima in features:
    img_file = cv2.imread(ima)
    img_file = resize(img_file, (imageSize, imageSize, 3))
    img_arr = np.asarray(img_file)
    data.append(img_arr)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, train_size=0.7)
print("Train data : ",len(X_train),len(y_train))
print("Validation data : ",len(X_test),len(y_test))

In [ ]:
X_train = np.array(X_train) 
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [9]:
del data, features, labels

In [10]:
input_dim = X_train[0].shape
input_dim

In [25]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(4, 4), activation='relu', input_shape=input_dim))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(64, kernel_size=(4, 4), strides=1, activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(128, kernel_size=(4, 4), strides=1, activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(n_classes, activation='softmax'))

opt = Adam(learning_rate=0.0001)
model.compile(optimizer=opt,loss="sparse_categorical_crossentropy",metrics=['accuracy'])

#opt = Adam(learning_rate=0.0001)
#model.compile(optimizer=opt,loss="sparse_categorical_crossentropy",metrics=['accuracy'])
model.summary()

In [26]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',patience=2)

In [27]:
history = model.fit(X_train,
          y_train,
          epochs=40,
          validation_data = (X_test,y_test),
          callbacks=[early_stop])

In [28]:
import matplotlib.pyplot as plt

plt.style.use('seaborn')
plt.figure(figsize=(10,10))
plt.plot(history.history['loss'], color='b', label="Training loss")
plt.plot(history.history['val_loss'], color='r', label="Validation loss")
plt.legend()
plt.show()

plt.figure()

plt.figure(figsize=(8,8))
plt.plot(history.history['accuracy'], color='b', label="Training accuracy")
plt.plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
plt.legend()
plt.show()

In [29]:
model.save('model2.h5')

In [17]:
!pip install tensorflowjs

In [18]:
!mkdir carpeta_salida

In [31]:
!tensorflowjs_converter --input_format keras model2.h5 carpeta_salida2

In [33]:
import shutil
shutil.make_archive('model_low2', 'zip', 'carpeta_salida2')

In [21]:
X_test[0].shape

In [22]:
examp = np.expand_dims(X_test[0], axis=0)
examp.shape

In [23]:
np.argmax(model.predict(examp,1))